[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/co-corporeality/tutorial-notebooks/blob/master/Train%20Classifier.ipynb)

In this notebook, you will learn how to train a ResNet34 classifier to recognize objects. After training the model by capturing photos from the webcam, it can be exported, run on your local machine, and connected to another application like Unity.

Mount gDrive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Install required Linux packages

In [0]:
!apt-get update && apt-get upgrade -y &&\
    apt-get autoremove -y
!sudo apt install -y --no-install-recommends \
      build-essential \
      cpio \
      curl \
      git \
      lsb-release \
      pciutils \
      python3.6 \
      python3-pip \
      sudo \
      libusb-1.0-0 libboost-program-options1.62.0 \
      libboost-thread1.62.0 libboost-filesystem1.62.0 \
      libssl1.0.0 libudev1 libjson-c3 usbutils udev wget

Install OpenVino

In [0]:
from google.colab import drive
openvino_path = "/content/drive/My Drive/openvino/l_openvino_toolkit.tgz" 
  
!tar xf "{openvino_path}" &&\
    cd l_openvino_toolkit_p* && \
    ./install_openvino_dependencies.sh &&\
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg  


Install OpenVino prerequisites

In [0]:
!/opt/intel/openvino/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites.sh

Import Python libraries

In [0]:
from fastai.vision import *
from pathlib import Path
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import os

def take_photos(image_class, quality=0.8):
  js = Javascript('''
    async function takePhotos(quality, imageClass) {
      const div = document.createElement('div');

      const header = document.createElement('div');
      const capture = document.createElement('button');
      const leftCounter = document.createElement('span');
      
      let toCapture = 30;

      header.innerHTML = '<h2>Capturing images for class <b>' + imageClass + '</b></h2><br/>';
      capture.textContent = 'Capture';
      leftCounter.innerHTML = toCapture + ' left.';
      
      div.appendChild(header);
      div.appendChild(capture);
      div.appendChild(leftCounter);

      const video = document.createElement('video');
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;

      const images = document.createElement('div');
      images.style.display = 'block';
      
      images.style.width = '100%';
      images.style.height = '200px';
      images.style.overflow = 'scroll';
      images.style.backgroundColor = 'gray';

      div.appendChild(images);

      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      let urls = [];
      const captureCanvas = document.createElement('canvas');

      let finished = {};

      capture.onclick = () => {
        captureCanvas.width = video.videoWidth;
        captureCanvas.height = video.videoHeight;          
        captureCanvas.getContext('2d').drawImage(video, 0, 0);
        urls.push(captureCanvas.toDataURL('image/jpeg', quality));

        const displayCanvas = document.createElement('canvas');
        displayCanvas.width = video.videoWidth / 5.0;
        displayCanvas.height = video.videoHeight / 5.0;
        images.appendChild(displayCanvas);
        let context = displayCanvas.getContext('2d');
        context.scale(1.0/5.0, 1.0/5.0);
        context.drawImage(captureCanvas, 0, 0);

        --toCapture;
        header.innerHTML = '<h2>Capturing images for class <b>' + imageClass + '</b></h2><br/>';
        leftCounter.innerHTML = toCapture + ' left.';

        if (toCapture == 0) {
            finished.call();
        }
      };

      await new Promise((resolve) => finished.call = resolve);
      
      stream.getVideoTracks()[0].stop();
      div.remove();
      
      return urls;
    }
    ''')
  display(js)
  datalist = eval_js('takePhotos({}, "{}")'.format(quality, image_class))
  try:
    os.rmdir('data')
  except:
    pass
  
  try:
    data_path = f'data/{image_class}'
    os.makedirs(data_path, exist_ok=True)
  except:
    print('Data path already exists.')
  
  i = 0
  filenames = []
  for data in datalist:
    binary = b64decode(data.split(',')[1])
    filename = f'{data_path}/photo{i}.jpg'
    with open(filename, 'wb') as f:
      f.write(binary)
    i += 1
    filenames.append(filename)
  
  return filenames

In [0]:
classes = ['apple', 'kiwi']

try:
  for c in classes:
    filenames = take_photos(c)
except Exception as err:
  print(str(err))

In [0]:
np.random.seed(42)
data = ImageDataBunch.from_folder('data', valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

In [0]:
data.show_batch(rows=3, figsize=(7,8))

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [0]:
learn.fit_one_cycle(4)

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.plot_confusion_matrix()

In [0]:
interp.plot_top_losses(10)

Export the trained model to ONNX

In [0]:
import torch
model = learn.model

dummy_input = torch.Tensor(torch.randn(1, 3, 224, 224)).to(device)

export_model = nn.Sequential(*(list(model.children()) + [nn.Softmax(dim=1)])) 

os.mkdir('/content/drive/My Drive/openvino_export/')
torch.onnx.export(export_model, dummy_input, '/content/drive/My Drive/openvino_export/model.onnx', input_names=['input'], output_names=['output'], verbose=False)

Convert the ONNX model to the OpenVino format

In [0]:
! source /opt/intel/openvino/bin/setupvars.sh
! python /opt/intel/openvino/deployment_tools/model_optimizer/mo.py --input_model /content/drive/My\ Drive/openvino_export/model.onnx --output_dir /content/drive/My\ Drive/openvino_export/ --mean_values "[123.675, 116.28, 103.53]" --scale_values "[58.395, 57.12, 57.375]" --reverse_input_channels